# LangChain: Evaluation

## Outline:

* Example generation
* Manual evaluation (and debuging)
* LLM-assisted evaluation
* LangChain evaluation platform

In [1]:
import os

from dotenv import load_dotenv, find_dotenv
_ = load_dotenv(find_dotenv()) # read local .env file

Note: LLM's do not always produce the same results. When executing the code in your notebook, you may get slightly different answers that those in the video.

In [2]:
# account for deprecation of LLM model
import datetime
# Get the current date
current_date = datetime.datetime.now().date()

# Define the date after which the model should be set to "gpt-3.5-turbo"
target_date = datetime.date(2024, 6, 12)

# Set the model variable based on the current date
if current_date > target_date:
    llm_model = "gpt-3.5-turbo"
else:
    llm_model = "gpt-3.5-turbo-0301"

## Create our QandA application

In [3]:
from langchain.chains import RetrievalQA
from langchain_openai import ChatOpenAI
from langchain.document_loaders import CSVLoader
from langchain.indexes import VectorstoreIndexCreator
from langchain_openai import OpenAIEmbeddings
from langchain.vectorstores import DocArrayInMemorySearch

In [4]:
from pathlib import Path

data_file = Path("data") / "OutdoorClothingCatalog_1000.csv"
loader = CSVLoader(file_path=str(data_file), encoding="utf-8")
data = loader.load()

In [5]:
# create embeddings and pass to the index creator
embeddings = OpenAIEmbeddings()
index = VectorstoreIndexCreator(
    vectorstore_cls=DocArrayInMemorySearch,
    embedding=embeddings,
).from_loaders([loader])

c:\Users\chaey\Desktop\OneDrive\Documents\GitHub\knowledge-sharing\.venv\Lib\site-packages\pydantic\_migration.py:283: UserWarning: `pydantic.error_wrappers:ValidationError` has been moved to `pydantic:ValidationError`.
  warnings.warn(f'`{import_path}` has been moved to `{new_location}`.')


In [6]:
llm = ChatOpenAI(temperature = 0.0, model=llm_model)
qa = RetrievalQA.from_chain_type(
    llm=llm, 
    chain_type="stuff", 
    retriever=index.vectorstore.as_retriever(), 
    verbose=True,
    chain_type_kwargs = {
        "document_separator": "<<<<>>>>>"
    }
)

### Coming up with test datapoints

In [7]:
data[10]

Document(metadata={'source': 'data\\OutdoorClothingCatalog_1000.csv', 'row': 10}, page_content=": 10\nname: Cozy Comfort Pullover Set, Stripe\ndescription: Perfect for lounging, this striped knit set lives up to its name. We used ultrasoft fabric and an easy design that's as comfortable at bedtime as it is when we have to make a quick run out.\n\nSize & Fit\n- Pants are Favorite Fit: Sits lower on the waist.\n- Relaxed Fit: Our most generous fit sits farthest from the body.\n\nFabric & Care\n- In the softest blend of 63% polyester, 35% rayon and 2% spandex.\n\nAdditional Features\n- Relaxed fit top with raglan sleeves and rounded hem.\n- Pull-on pants have a wide elastic waistband and drawstring, side pockets and a modern slim leg.\n\nImported.")

In [8]:
data[11]

Document(metadata={'source': 'data\\OutdoorClothingCatalog_1000.csv', 'row': 11}, page_content=': 11\nname: Ultra-Lofty 850 Stretch Down Hooded Jacket\ndescription: This technical stretch down jacket from our DownTek collection is sure to keep you warm and comfortable with its full-stretch construction providing exceptional range of motion. With a slightly fitted style that falls at the hip and best with a midweight layer, this jacket is suitable for light activity up to 20° and moderate activity up to -30°. The soft and durable 100% polyester shell offers complete windproof protection and is insulated with warm, lofty goose down. Other features include welded baffles for a no-stitch construction and excellent stretch, an adjustable hood, an interior media port and mesh stash pocket and a hem drawcord. Machine wash and dry. Imported.')

### Hard-coded examples

In [9]:
examples = [
    {
        "query": "Do the Cozy Comfort Pullover Set\
        have side pockets?",
        "answer": "Yes"
    },
    {
        "query": "What collection is the Ultra-Lofty \
        850 Stretch Down Hooded Jacket from?",
        "answer": "The DownTek collection"
    }
]

### LLM-Generated examples

In [10]:
from langchain.evaluation.qa import QAGenerateChain

In [11]:
example_gen_chain = QAGenerateChain.from_llm(ChatOpenAI(model=llm_model))

In [13]:
new_examples = example_gen_chain.batch(
    [{"doc": t} for t in data[:5]]
)

In [14]:
new_examples[0]

{'doc': Document(metadata={'source': 'data\\OutdoorClothingCatalog_1000.csv', 'row': 0}, page_content=": 0\nname: Women's Campside Oxfords\ndescription: This ultracomfortable lace-to-toe Oxford boasts a super-soft canvas, thick cushioning, and quality construction for a broken-in feel from the first time you put them on. \n\nSize & Fit: Order regular shoe size. For half sizes not offered, order up to next whole size. \n\nSpecs: Approx. weight: 1 lb.1 oz. per pair. \n\nConstruction: Soft canvas material for a broken-in feel and look. Comfortable EVA innersole with Cleansport NXT® antimicrobial odor control. Vintage hunt, fish and camping motif on innersole. Moderate arch contour of innersole. EVA foam midsole for cushioning and support. Chain-tread-inspired molded rubber outsole with modified chain-tread pattern. Imported. \n\nQuestions? Please contact us for any inquiries."),
 'qa_pairs': {'query': "What is the approximate weight of the Women's Campside Oxfords per pair?",
  'answer': 

In [15]:
data[0]

Document(metadata={'source': 'data\\OutdoorClothingCatalog_1000.csv', 'row': 0}, page_content=": 0\nname: Women's Campside Oxfords\ndescription: This ultracomfortable lace-to-toe Oxford boasts a super-soft canvas, thick cushioning, and quality construction for a broken-in feel from the first time you put them on. \n\nSize & Fit: Order regular shoe size. For half sizes not offered, order up to next whole size. \n\nSpecs: Approx. weight: 1 lb.1 oz. per pair. \n\nConstruction: Soft canvas material for a broken-in feel and look. Comfortable EVA innersole with Cleansport NXT® antimicrobial odor control. Vintage hunt, fish and camping motif on innersole. Moderate arch contour of innersole. EVA foam midsole for cushioning and support. Chain-tread-inspired molded rubber outsole with modified chain-tread pattern. Imported. \n\nQuestions? Please contact us for any inquiries.")

### Combine examples

In [16]:
examples += new_examples

In [17]:
qa.invoke({"query": examples[0]["query"]})



> Entering new RetrievalQA chain...

> Finished chain.


{'query': 'Do the Cozy Comfort Pullover Set        have side pockets?',
 'result': 'Yes, the Cozy Comfort Pullover Set does have side pockets.'}

## Manual Evaluation

In [18]:
import langchain
langchain.debug = True

In [19]:
qa.run(examples[0]["query"])

[chain/start] [chain:RetrievalQA] Entering Chain run with input:
{
  "query": "Do the Cozy Comfort Pullover Set        have side pockets?"
}


C:\Users\chaey\AppData\Local\Temp\ipykernel_29156\1223946598.py:1: LangChainDeprecationWarning: The method `Chain.run` was deprecated in langchain 0.1.0 and will be removed in 1.0. Use :meth:`~invoke` instead.
  qa.run(examples[0]["query"])


[chain/start] [chain:RetrievalQA > chain:StuffDocumentsChain] Entering Chain run with input:
[inputs]
[chain/start] [chain:RetrievalQA > chain:StuffDocumentsChain > chain:LLMChain] Entering Chain run with input:
{
  "question": "Do the Cozy Comfort Pullover Set        have side pockets?",
  "context": ": 10\nname: Cozy Comfort Pullover Set, Stripe\ndescription: Perfect for lounging, this striped knit set lives up to its name. We used ultrasoft fabric and an easy design that's as comfortable at bedtime as it is when we have to make a quick run out.\n\nSize & Fit\n- Pants are Favorite Fit: Sits lower on the waist.\n- Relaxed Fit: Our most generous fit sits farthest from the body.\n\nFabric & Care\n- In the softest blend of 63% polyester, 35% rayon and 2% spandex.\n\nAdditional Features\n- Relaxed fit top with raglan sleeves and rounded hem.\n- Pull-on pants have a wide elastic waistband and drawstring, side pockets and a modern slim leg.\n\nImported.<<<<>>>>>: 73\nname: Cozy Cuddles Knit

'Yes, the Cozy Comfort Pullover Set does have side pockets.'

In [20]:
# Turn off the debug mode
langchain.debug = False

## LLM assisted evaluation

In [21]:
# ensure every input dict has the 'query' key the RetrievalQA chain expects
predictions = []
for ex in examples:
    q = ex.get("query")
    if not q:
        predictions.append({"query": None, "result": None})
        continue
    result = qa.run(q)
    predictions.append({"query": q, "result": result})



> Entering new RetrievalQA chain...

> Finished chain.


> Entering new RetrievalQA chain...

> Finished chain.


In [22]:
from langchain.evaluation.qa import QAEvalChain

In [23]:
llm = ChatOpenAI(temperature=0, model=llm_model)
eval_chain = QAEvalChain.from_llm(llm)

In [24]:
# Normalize examples & predictions so QAEvalChain.evaluate finds 'query','answer','result'
def _get(d, keys, default=None):
    for k in keys:
        if isinstance(d, dict) and k in d:
            return d[k]
    return default

canonical_examples = []
for ex in examples:
    if isinstance(ex, dict):
        q = _get(ex, ["query", "question", "q", "prompt"])
        a = _get(ex, ["answer", "answers", "label", "labels"]) or ""
    elif isinstance(ex, str):
        q = ex
        a = ""
    else:
        q = None
        a = ""
    canonical_examples.append({"query": q, "answer": a})

canonical_predictions = []
for i, ex in enumerate(canonical_examples):
    pred_item = {}
    if i < len(predictions) and isinstance(predictions[i], dict):
        pred_item["result"] = _get(predictions[i], ["result", "prediction", "answer", "text", "response"])
        pred_item["query"] = _get(predictions[i], ["query", "question"]) or ex["query"]
    else:
        pred_item["result"] = None
        pred_item["query"] = ex["query"]
    canonical_predictions.append(pred_item)

# now evaluate
graded_outputs = eval_chain.evaluate(canonical_examples, canonical_predictions)
print("First graded output:", graded_outputs[0] if graded_outputs else None)

First graded output: {'results': 'CORRECT'}


In [25]:
# Use canonical_examples / canonical_predictions / graded_outputs (not raw predictions) when printing
for i, ex in enumerate(canonical_examples):
    q = canonical_predictions[i].get("query") if i < len(canonical_predictions) else ex.get("query", "")
    real_a = ex.get("answer", "")
    pred_a = canonical_predictions[i].get("result") if i < len(canonical_predictions) else ""
    grade = graded_outputs[i].get("text") if graded_outputs and i < len(graded_outputs) else ""
    print(f"Example {i}:")
    print("Question: " + (q or ""))
    print("Real Answer: " + (real_a or ""))
    print("Predicted Answer: " + (pred_a or ""))
    print("Predicted Grade: " + (grade or ""))
    print()

Example 0:
Question: Do the Cozy Comfort Pullover Set        have side pockets?
Real Answer: Yes
Predicted Answer: Yes, the Cozy Comfort Pullover Set does have side pockets.
Predicted Grade: 

Example 1:
Question: What collection is the Ultra-Lofty         850 Stretch Down Hooded Jacket from?
Real Answer: The DownTek collection
Predicted Answer: The Ultra-Lofty 850 Stretch Down Hooded Jacket is from the DownTek collection.
Predicted Grade: 

Example 2:
Question: 
Real Answer: 
Predicted Answer: 
Predicted Grade: 

Example 3:
Question: 
Real Answer: 
Predicted Answer: 
Predicted Grade: 

Example 4:
Question: 
Real Answer: 
Predicted Answer: 
Predicted Grade: 

Example 5:
Question: 
Real Answer: 
Predicted Answer: 
Predicted Grade: 

Example 6:
Question: 
Real Answer: 
Predicted Answer: 
Predicted Grade: 



In [26]:
graded_outputs[0]

{'results': 'CORRECT'}